In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_io, hyspec_ml, image_render

In [3]:
# Paths
hyspec_dir = pathlib.Path(r'/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/2_R_rs')
pca_model_path = pathlib.Path(r'/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/M_PcaModelImage_13_20_21_24_25_30_31/pca_model.npz')
labels_dir = pathlib.Path(r'/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/M_OverClusteringAnnotation/all_gray')
train_dataset_path = pathlib.Path(r'/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/3_PcaDatasets/train')
val_dataset_path = pathlib.Path(r'/media/mha114/Massimal/Bodo_Juvika/Hyperspectral/20220624/Area/3_PcaDatasets/validation')


In [4]:
# Parameters
training_image_numbers = [13,20,21,24,25,30,31]
validation_image_numbers = [15,19,26,29]

In [5]:
# Load saved PCA model
data = {}
with np.load(pca_model_path) as npz_files:
    for variable_name in npz_files.files:
        print(variable_name)
        data[variable_name] = npz_files[variable_name]

X_mean = data['X_mean']
X_std = data['X_std']
W_pca = data['pca_components']

X_mean
X_std
pca_components


In [6]:
def labeled_images_to_pca_dataset(hyspec_dir,labels_dir,dataset_save_path,image_numbers,W_pca,X_mean,X_std):
    
    # Lists for collecting data
    pca_image_list = []
    label_image_list = []
    dataset_name_list = []

    for im_num in tqdm.tqdm(image_numbers):
        hyspec_path = list(hyspec_dir.glob('*_'+str(im_num)+'-*.hdr'))[0]
        label_path = list(labels_dir.glob('*_'+str(im_num)+'*'))[0]

        hyspec_image,_,_,_ = hyspec_io.load_envi_image(str(hyspec_path))
        pca_image = hyspec_ml.pca_transform_image(hyspec_image,W_pca,X_mean,X_std=X_std)
        pca_image_list.append(pca_image)
        zero_mask = np.all(pca_image==0,axis=2)

        label_image = skimage.io.imread(label_path)
        label_image[zero_mask] = 0
        label_image_list.append(label_image)

        dataset_name_list.append(hyspec_path.stem.split(sep='-')[0])
    
    # Save dataset
    dataset = tf.data.Dataset.from_tensor_slices((pca_image_list,label_image_list,dataset_name_list))
    tf.data.experimental.save(dataset,str(dataset_save_path))

In [7]:
labeled_images_to_pca_dataset(hyspec_dir,labels_dir,train_dataset_path,training_image_numbers,W_pca,X_mean,X_std)

100%|█████████████████████████████████████████████| 7/7 [02:24<00:00, 20.69s/it]


In [8]:
labeled_images_to_pca_dataset(hyspec_dir,labels_dir,val_dataset_path,validation_image_numbers,W_pca,X_mean,X_std)

100%|█████████████████████████████████████████████| 4/4 [02:29<00:00, 37.31s/it]
